# Example Sample Hotel and Flight Booker Agent 

This solution will help you book flight tickets and hotel.  The scenario is a trip London Heathrow LHR Feb 20th 2024 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel.


# Azure AI Agent Service प्रारंभ करा आणि **.env** मधून कॉन्फिगरेशन माहिती मिळवा

### **.env**

एक .env फाइल तयार करा

**.env** मध्ये Azure AI Agent Service चा कनेक्शन स्ट्रिंग, AOAI वापरलेला मॉडेल, आणि संबंधित Google API Search सेवा API, ENDPOINT इत्यादी असतात.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "आपल्या Azure AI Agent Service मॉडेल डिप्लॉयमेंटचे नाव"

[**NOTE**] आपल्याला 100,000 रेट लिमिट (टोकन्स प्रति मिनिट) आणि 600 रेट लिमिट (रिक्वेस्ट प्रति मिनिट) असलेला मॉडेल आवश्यक आहे

  तुम्ही Microsoft Foundry - Model आणि Endpoint मध्ये मॉडेल मिळवू शकता.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "आपल्या Azure AI Agent Service प्रोजेक्ट कनेक्शन स्ट्रिंग"

  तुम्ही AI Foundry Portal स्क्रीनच्या प्रोजेक्ट अवलोकनात प्रोजेक्ट कनेक्शन स्ट्रिंग मिळवू शकता.

- **SERPAPI_SEARCH_API_KEY** = "आपला SERPAPI Search API KEY"
- **SERPAPI_SEARCH_ENDPOINT** = "आपला SERPAPI Search Endpoint"

Azure AI Agent Service चे मॉडेल डिप्लॉयमेंट नाव आणि प्रोजेक्ट कनेक्शन स्ट्रिंग मिळवण्यासाठी, तुम्हाला Azure AI Agent Service तयार करावा लागेल. ते थेट तयार करण्यासाठी [हा टेम्प्लेट](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) वापरण्याची शिफारस केली जाते （***टीप:*** Azure AI Agent Service सध्या मर्यादित प्रदेशात सेट केलेले आहे. प्रदेश सेट करण्यासाठी [हा लिंक](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) पाहण्याची शिफारस आहे）

एजंटला SERPAPI कडे प्रवेश आवश्यक आहे. नोंदणीसाठी [हा लिंक](https://serpapi.com/searches) वापरण्याची शिफारस केली जाते. नोंदणीनंतर, तुम्हाला एक अद्वितीय API KEY आणि ENDPOINT मिळेल.


# सेटअप 

हा नोटबुक चालवण्यासाठी, तुम्हाला आवश्यक असलेल्या लायब्ररी इंस्टॉल केल्या आहेत याची खात्री करावी लागेल, त्यासाठी `pip install -r requirements.txt` चालवा.


In [ ]:
from semantic_kernel import __version__

__version__

आपली Semantic Kernel आवृत्ती किमान 1.27.2 असावी.


आपले .env फाइल सेटिंग आणि संसाधने लोड करा कृपया खात्री करा की आपण आपले कीज आणि सेटिंग्ज जोडले आहेत आणि स्थानिक .env फाइल तयार केली आहे.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Azure मध्ये लॉगिन करा

आपल्याला आता Azure मध्ये लॉगिन करावे लागेल. टर्मिनल उघडा आणि खालील आज्ञा चालवा:

```bash
az login
```

ही आज्ञा आपल्याला Azure क्रेडेन्शियल्स प्रविष्ट करण्यास सांगेल, ज्यामुळे Azure AI Agent सेवा योग्यरित्या कार्य करू शकेल.


# स्पष्टीकरण:
ही एक चल आहे जी SERP (शोध इंजिन परिणाम पृष्ठ) API सेवेसाठी API की साठवते. API की ही एक अद्वितीय ओळख आहे जी आपल्या खात्यातील विनंत्या प्रमाणित करण्यासाठी वापरली जाते.

उद्दिष्ट: या ओळीचा उद्देश API की एका चलात साठवणे हा आहे जेणेकरून ती SERP API सेवेला विनंत्या प्रमाणित करण्यासाठी वापरली जाऊ शकते. सेवा वापरण्यासाठी आणि शोध करण्यासाठी API की आवश्यक आहे.
SERP API की कशी मिळवायची: SERP API की मिळवण्यासाठी, https://serpapi.com या संकेतस्थळावर खालील सामान्य टप्पे फॉलो करा (विशिष्ट SERP API सेवेनुसार टप्पे वेगळे असू शकतात):

SERP API सेवा निवडा: अनेक SERP API सेवा उपलब्ध आहेत, जसे की SerpAPI, Google Custom Search JSON API, आणि अन्य. आपल्या गरजेनुसार योग्य सेवा निवडा.

खाते तयार करा: निवडलेली SERP API सेवा असलेल्या वेबसाइटवर जा आणि खाते तयार करा. कदाचित काही मूलभूत माहिती प्रदान करावी लागेल आणि आपला ईमेल पत्ता पुष्टी करावा लागेल.

API की तयार करा: खाते बनवल्यानंतर, आपले खाते लॉगिन करा आणि API विभाग किंवा डॅशबोर्डमध्ये जा. नवीन API की तयार करण्याचा किंवा जनरेट करण्याचा पर्याय शोधा.
API की आपल्या .env फाईलमध्ये कॉपी करा.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# स्पष्टीकरण:
BASE_URL: हा एक व्हेरिएबल आहे जो SERP API एंडपॉइंटसाठी बेस URL साठवतो. BASE_URL हा व्हेरिएबल नाव असा संकेत देतो की हा URL API विनंत्या करण्यासाठी सुरुवातीचा बिंदू आहे.
'https://serpapi.com/search':

हा BASE_URL व्हेरिएबलला असाइन केलेला खरा URL स्ट्रिंग आहे. तो SERP API वापरून शोध क्वेरीजसाठी एंडपॉइंटचे प्रतिनिधित्व करतो.

# उद्देश:
या ओळीचा उद्देश हा आहे की तो एक स्थिरांक परिभाषित करतो जो SERP API साठी बेस URL धारण करतो. हा URL API विनंत्या तयार करण्यासाठी सुरुवातीचा बिंदू म्हणून वापरला जाईल ज्यामुळे शोध ऑपरेशन्स करता येतील.

# वापर:
बेस URL एका व्हेरिएबलमध्ये परिभाषित करून, तुम्ही तो तुमच्या कोडमध्ये सहज पुन्हा वापरू शकता जेव्हा तुम्हाला SERP API कडे विनंत्या करायच्या असतील. यामुळे तुमचा कोड अधिक देखभालयोग्य होतो आणि URL अनेक ठिकाणी हार्डकोड केल्यामुळे होणारी चूक कमी होते. सध्याचा उदाहरण https://serpapi.com/search?engine=bing आहे जो Bing शोध API वापरत आहे. वेगवेगळ्या API निवडण्यासाठी https://Serpapi.com वर जाऊ शकता.


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# स्पष्टीकरण:

आपला प्लगइन कोड येथे आहे.

क्लास व्याख्या: `class BookingPlugin`: BookingPlugin नावाचा एक वर्ग परिभाषित करतो ज्यामध्ये हॉटेल्स आणि फ्लाइट्स बुक करण्यासाठी पद्धती असतात.

हॉटेल बुकिंग पद्धत:

- `@kernel_function(description="booking hotel")`: हा डेकोरेटर फंक्शनला हॉटेल बुकिंगसाठी कर्नेल फंक्शन म्हणून वर्णन करतो.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: अॅनोटेट केलेल्या पॅरामीटर्स आणि रिटर्न टाइपसह हॉटेल्स बुक करण्यासाठी एक पद्धत परिभाषित करतो.

ही पद्धत हॉटेल बुकिंग विनंतीसाठी पॅरामीटर्सचा एक डिक्शनरी तयार करते आणि SERP API ला GET विनंती पाठवते. ती प्रतिसाद स्थिती तपासते आणि यशस्वी झाल्यास हॉटेलची माहिती परत करते, अन्यथा विनंती अयशस्वी झाली असल्यास None परत करते.

फ्लाइट बुकिंग पद्धत:

- `@kernel_function(description="booking flight")`: हा डेकोरेटर फंक्शनला फ्लाइट बुकिंगसाठी कर्नेल फंक्शन म्हणून वर्णन करतो.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: अॅनोटेट केलेल्या पॅरामीटर्स आणि रिटर्न टाइपसह फ्लाइट्स बुक करण्यासाठी एक पद्धत परिभाषित करतो.

ही पद्धत आउटबाउंड आणि परतीच्या फ्लाइट विनंत्यांसाठी पॅरामीटर्सचा डिक्शनरी तयार करते आणि SERP API ला GET विनंत्या पाठवते. ती प्रतिसाद स्थिती तपासते आणि यशस्वी झाल्यास फ्लाइट माहिती परिणाम स्ट्रिंगमध्ये जोडते, अन्यथा विनंती अयशस्वी झाल्यास त्रुटी संदेश मुद्रित करते. ही पद्धत फ्लाइट माहितीसह परिणाम स्ट्रिंग परत करते.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# स्पष्टीकरण:
आयात विधान: Azure क्रेडेन्शियल, AI एजंट, चॅट मेसेज सामग्री, लेखकाची भूमिका, आणि कर्नेल फंक्शन डेकोरेटरसाठी आवश्यक मॉड्युल आयात करा.

असिंक्रोनस संदर्भ व्यवस्थापक: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): हे Azure क्रेडेन्शियल हाताळण्यासाठी आणि AI एजंट क्लायंट तयार करण्यासाठी असिंक्रोनस संदर्भ व्यवस्थापक सेट करते.

एजंटचे नाव आणि सूचना:
- `AGENT_NAME = "BookingAgent"`: एजंटचे नाव परिभाषित करते.
- `AGENT_INSTRUCTIONS = """..."""`: बुकिंग विनंत्या कशा हाताळायच्या याबद्दल एजंटसाठी सविस्तर सूचना प्रदान करतो.

एजंट परिभाषा तयार करा: `agent_definition = await client.agents.create_agent(...)`: निर्दिष्ट मॉडेल, नाव, आणि सूचना सह एजंट परिभाषा तयार करतो.

AzureAI एजंट तयार करा: `agent = AzureAIAgent(...)`: क्लायंट, एजंट परिभाषा, आणि परिभाषित प्लगइन वापरून AzureAI एजंट तयार करतो.

थ्रेड तयार करा: `thread: AzureAIAgentThread | None = None`: एजंटसाठी थ्रेड तयार करा. प्रथम थ्रेड तयार करणे आवश्यक नाही - जर `None` चे मूल्य दिले गेले, तर प्रथम कॉल दरम्यान नवीन थ्रेड तयार केला जाईल आणि प्रतिसादाचा भाग म्हणून परत केला जाईल.

वापरकर्त्याची इनपुट्स: `user_inputs = ["..."]`: एजंटसाठी प्रक्रिया करण्यासाठी वापरकर्त्याच्या इनपुट्सची यादी परिभाषित करते.

finally ब्लॉकमध्ये, संसाधने स्वच्छ करण्यासाठी थ्रेड आणि एजंट हटवा.


# प्रमाणीकरण

`DefaultAzureCredential` वर्ग Azure SDK for Python चा भाग आहे. हे Azure सेवांसह प्रमाणीकरण करण्याचा डीफॉल्ट मार्ग प्रदान करते. हे विशिष्ट क्रमाने एकाधिक पद्धती वापरून प्रमाणीकरण करण्याचा प्रयत्न करते, जसे की पर्यावरण चल, व्यवस्थापित ओळख, आणि Azure CLI क्रेडेन्शियल्स.

असिंक्रोनस ऑपरेशन्स: aio मॉड्यूल हे सूचित करते की DefaultAzureCredential वर्ग असिंक्रोनस ऑपरेशन्सना समर्थन करतो. याचा अर्थ तुम्ही हे asyncio सह वापरू शकता जेणेकरून प्रतिबंधित न करता प्रमाणीकरण विनंत्या पार पाडता येतील.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**सूचना**:
हा दस्तऐवज AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) वापरून अनुवादित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्न करतो, परंतु कृपया लक्षात घ्या की स्वयंचलित अनुवादांमध्ये त्रुटी किंवा अचूकतेची कमतरता असू शकते. मूळ दस्तऐवज त्याच्या मातृभाषेत अधिकृत स्त्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी अनुवादाची शिफारस केली आहे. या अनुवादाच्या वापरामुळे उद्भवणाऱ्या कोणत्याही गैरसमजुती किंवा चुकीच्या अर्थनिर्णयाबद्दल आम्ही जबाबदार नाही.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
